In [ ]:
Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

если источник traffic_source равен yandex или google, то в source_type ставится organic
для источников paid и email из России - ставим ad
для источников paid и email не из России - ставим other
все остальные варианты берем из traffic_source без изменений


In [1]:
import pandas as pd

In [2]:
logs = pd.read_csv('visit_log.csv', sep=';')

In [3]:
def traffic_type(df):
    if df.traffic_source in ['yandex','google']:
        return 'organic'
    elif (df.traffic_source in ['paid','email']) & (df.region == 'Russia'):
        return 'ad'
    elif (df.traffic_source in ['paid','email']) & (df.region != 'Russia'):
        return 'other'
    else:
        return df.traffic_source

In [4]:
logs['source_type'] = logs.apply(traffic_type, axis = 1)

In [5]:
logs.source_type.value_counts()

direct     7013
organic    5407
ad         3790
other      2728
Name: source_type, dtype: int64

In [ ]:
Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами
новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис.
    Выполните следующие действия:

Прочитайте содержимое файла с датафрейм
Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии 
с заданным шаблоном


In [6]:
import regex as re

In [7]:
urls = pd.read_csv('URLs.txt')

In [8]:
urls[(urls.url.str.contains(r'\/\d{8}\-'))]

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
...,...
88,/cis/35984145-kreml-prokommentiroval-soobschen...
89,/video/36071019-olimpiyskie-obekty-rio-prevrat...
90,/science/36151301-nazvano-posledstvie-zloupotr...
91,/incidents/36027330-vospitatelnitsu-zatravili-...


In [9]:
len(urls[(urls.url.str.contains(r'\/\d{8}\-'))])

84

In [ ]:
Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, 
которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца 
timestamp для данного значения userId.


In [10]:
ratings = pd.read_csv('ratings.csv')

In [11]:
our_users = ratings.groupby('userId').count()['rating'] > 100

In [12]:
our_users = pd.DataFrame(our_users[our_users == True])

In [13]:
our_users

,rating
userId,
4,True
8,True
15,True
17,True
19,True
...,...
656,True
659,True
664,True


In [14]:
min_max_users = ratings.groupby('userId').agg(['min', 'max'])['timestamp']

In [15]:
min_max_users

,min,max
userId,,
1,1260759108,1260759205
2,835355395,835356246
3,1298861589,1298932787
4,949778714,949982274
5,1163373044,1163375145
...,...,...
667,847271220,847272234
668,993613196,993613478
669,1015829081,1015829766


In [16]:
min_max_users = our_users.join(min_max_users)

In [17]:
min_max_users

,rating,min,max
userId,,,
4,True,949778714,949982274
8,True,1154389340,1154474527
15,True,997937239,1469330735
17,True,1127468587,1127476640
19,True,855190091,855195373
...,...,...,...
656,True,986240991,986244044
659,True,834598040,866207451
664,True,1343731283,1441911722


In [18]:
min_max_users['min_max'] = min_max_users['max'] - min_max_users['min']

In [19]:
min_max_users

,rating,min,max,min_max
userId,,,,
4,True,949778714,949982274,203560
8,True,1154389340,1154474527,85187
15,True,997937239,1469330735,471393496
17,True,1127468587,1127476640,8053
19,True,855190091,855195373,5282
...,...,...,...,...
656,True,986240991,986244044,3053
659,True,834598040,866207451,31609411
664,True,1343731283,1441911722,98180439


In [20]:
min_max_users.min_max.mean()

40080507.4496124

In [ ]:
Задание 4

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента

In [21]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [22]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [23]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [24]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [25]:
summary_rev = rzd.merge(auto, how = 'outer')

In [26]:
summary_rev = summary_rev.merge(air, how = 'outer')

In [27]:
summary_rev

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [28]:
summary_rev2 = summary_rev.merge(client_base, how = 'outer')

In [29]:
summary_rev2

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4
